In [4]:
import glob
import pandas as pd

In [32]:
# Read all csv files in the data folder and subfolders
data_files = glob.glob('data/**/IVg*.csv', recursive=True)
data = [pd.read_csv(f, comment='#') for f in data_files]

In [38]:
def get_params(file: str):
    """Get the parameters from the file header. Return a dictionary with the
    parameters.
    """
    dic = dict()
    with open(file) as f:
        for line in f:
            if not line.startswith('#'):
                break

            key, value = line.strip('#').strip().split(':')
            if not key in ['Parameters', 'Data']:
                dic[key] = value.strip()
            if key == 'Procedure':
                # remove procedure from '<__main__.procedure>'
                procedure = value.strip().split('.')[-1][:-1]
                dic[key] = procedure
                dic['Date'] = file.split(procedure)[-1][:10]
    return dic

In [39]:
params = [get_params(f) for f in data_files]

In [40]:
params[-3]

{'Procedure': 'IVg',
 'Date': '2023-05-12',
 'Irange': '0.001 A',
 'N_avg': '2',
 'Chip': 'Unknown',
 'Comment': '-',
 'Sample': 'Unknown',
 'Step time': '0.01 s',
 'VDS': '0.075 V',
 'VG end': '35 V',
 'VG start': '-35 V',
 'VG step': '0.2 V'}

In [35]:
data[-3]

,Vg (V),I (A),hola
